

<hr style="height:5px;border-width:0;color:orange;background-color:orange">


<hr style="height:5px;border-width:0;color:orange;background-color:orange">

<center> <span style="font-size: 48px;"> Biomedical RAG Chatbot - GenAI </span> </center>

<p align="center">
    <img src="images/genai_chat.png" alt="Descripción de la imagen" width="60%">
</p>

+ Author: **Alejandro Sánchez Silvestre**  
+ Date: **Dic 2024**  
+ Module: **GenAI**

<hr style="height:5px;border-width:0;color:orange;background-color:orange">


<hr style="height:5px;border-width:0;color:orange;background-color:orange">


<a id='seccion1'></a>
# **Intro**

I've build this RAG chatbot to provide support to a LLM with an especific acurate context out of the training of the LLM in order the model to be able to answer new specialized questions about a certain topic.

Three books has been processed and loaded to feed the RAG system.

- **Pharmacology:** Rang & Dale's Pharmacology, 9th Edition (2020)

- **Medicine:** Harrison's Manual of Medicine, 20th Edition (2018)

- **Toxicology:** Principes of Toxicology Stine & Brown, 3rd Edition(2015)

In total more than 2500 pages of an especific field information.


<p align="center">
    <img src="images/books.png" alt="Descripción de la imagen" width="70%">
</p>



The aim of creating this project was to create a specialized RAG Chatbot overcomming the challenge of answering domain-specific questions with coherence and precission.

**Outline of the project**

1. Requirements and Libraries
2. Document loading
3. Document Processing & Spliting
4. Embeddings generation using Specialized Biomedical Domain Model
5. RAG construction
6. Building the LLM for generating the answer
7. Testing
8. Building the RAG-Chain & Prompt Engineering
9. Conclusion





# **1. Requirements & Libraries**
<hr style="height:5px;border-width:0;color:orange;background-color:orange">

First install the requirements and libraries needed.

In [2]:
!pip install -r requirements.txt --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 131.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 58.7 MB/s eta

In [3]:
# Core Python Libraries
import os
from dotenv import load_dotenv
import getpass
import re
import random

# Google Colab Utilities
from google.colab import drive

# Web and File Parsing
import bs4
from langchain_community.document_loaders import WebBaseLoader

# LangChain Core Libraries
from langchain import hub
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Hugging Face Libraries
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

# Sentence Transformers
from sentence_transformers import SentenceTransformer

# PyTorch
import torch

# Semantic Chunking
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.document_loaders import PyPDFLoader

# Prompt Engineering
from langchain.prompts.chat import ChatPromptTemplate

import warnings
warnings.filterwarnings("ignore")


In [5]:
#get the secret keys from the .env file
os.environ["LANGCHAIN_TRACING_V2"] = "true"
load_dotenv()

True

In [6]:
#Check api keys
api_keys = {
    "OPENAI_API_KEY": "Enter your OpenAI API key: ",
    "PINECONE_API_KEY": "Enter your Pinecone API key: ",
    "LANGCHAIN_API_KEY": "Enter your LangChain API key: ",
    "HF_TOKEN": "Enter your Hugging Face token: ",
}

# Config api key in case of not loaded
for key, prompt in api_keys.items():
    if not os.environ.get(key):  # Solo solicita la clave si no está configurada
        os.environ[key] = getpass.getpass(prompt)

# Check the API Keys correctly loaded
for key in api_keys.keys():
    print(f"{key} is set: {'Yes' if os.environ.get(key) else 'No'}")
print('--\n')


# Get hugging face token
hf_token = os.getenv("HF_TOKEN")

# Log in using the token
login(hf_token)
print("Logged into Hugging Face successfully!")


OPENAI_API_KEY is set: Yes
PINECONE_API_KEY is set: Yes
LANGCHAIN_API_KEY is set: Yes
HF_TOKEN is set: Yes
--



Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Logged into Hugging Face successfully!


# **2. Document loading**
<hr style="height:5px;border-width:0;color:orange;background-color:orange">

As mentioned above the documents contain specific medical domain information.

Here below a sample screenshot from the Pharmacology book (Rang & Dale's).
Notice the high specialized lexic such as ''antipyretic'', ''NSAID'' or ''cyclo-oxygenase inhibitory action''.

Words that for a non-healthcare professional probably are unkown.

This set up an extra challenge of finding a correct model for a good understanding of the corpus text.


<p align="center">
    <img src="images/paracetamol.png" alt="Descripción de la imagen" width="60%">
</p>

In [8]:
# DOCUMENT LOADING
# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Path to the folder in Google Drive containing your PDF files
drive_folder_path = '/content/drive/My Drive/Data_Science_Repo_Data/LLM_Chatbot'

# Step 2: List all PDF files in the folder
pdf_files = [file for file in os.listdir(drive_folder_path) if file.endswith('.pdf')]
print(f"Found {len(pdf_files)} PDF files: {pdf_files}")
pdf_files

Mounted at /content/drive
Found 3 PDF files: ['EN_rang-and-dales-pharmacology.pdf', 'EN_principles-of-toxicology-third-edition.pdf', 'EN_harrisons-manual-of-medicine.pdf']


['EN_rang-and-dales-pharmacology.pdf',
 'EN_principles-of-toxicology-third-edition.pdf',
 'EN_harrisons-manual-of-medicine.pdf']

In [9]:

# Step 3: Load and process the docs using LangChain

docs = []
for pdf_file in pdf_files:
    pdf_path = os.path.join(drive_folder_path, pdf_file)
    loader = PyPDFLoader(pdf_path)

    # Load documents from the PDF
    pdf_docs = loader.load()

    # Add metadata to each document
    for i, doc in enumerate(pdf_docs):
        doc.metadata = {
            "source": pdf_file,  # The file name
            "page": i + 1        # Page number (1-based index)
        }
    # Append processed docs with metadata
    docs.extend(pdf_docs)

print(f"Loaded {len(docs)} docs.")


Loaded 2470 docs.


It's important to understand the LLM will generate most likely anything relate but not accurate or true when hallucinating.

For this purpose and to test the RAG it's introduced information about a recent event out of the training scope of the model.

For this reason I chosed a text about Trump victory un US Presidential election that was celebrated last november 2024.

In [10]:
# Step 4, inject some text of recent event to test and validating the RAG with information post-training for the LLM(September 27, 2023)


test_text = """
In a surprising turn of events, Donald J. Trump won the 2024 US Presidential Elections.
Running as the Republican candidate, Trump secured 290 electoral votes, defeating the Democratic candidate,
Kamala Harris, who garnered 248 electoral votes.The victory was largely attributed to high voter turnout
in swing states such as Pennsylvania, Michigan, and Arizona. Trump's campaign focused on economic recovery,
with promises to lower inflation and bolster manufacturing jobs in the US.

Key moments from the campaign included a record-breaking rally in Texas and a strong debate
performance where Trump emphasized his "America First" policies. On the other hand, Harris
focused on healthcare reform and climate change but struggled to gain traction in key battleground states.

Following the election, Trump announced plans to immediately implement tax reforms and revisit foreign
trade agreements. The results sparked mixed reactions across the country, with celebrations in Republican strongholds
 and protests in urban Democratic areas.

The 2024 elections saw the highest voter turnout in over 120 years, highlighting the deep political engagement of the American populace.
"""

# Create a LangChain Document object for the Trump text
test_doc = Document(page_content=test_text, metadata={"source": "Generated Text - Trump 2024"})

# Step 3: Add the Trump document to the existing docs
docs.append(test_doc)

print(f"Loaded {len(docs)} docs, including the Trump text test.")


Loaded 2471 docs, including the Trump text test.


In [11]:
# Check for the correct document loading

# print random doc
if docs:
    random_doc = random.choice(docs)
    print(random_doc.page_content[:2000])
    print(random_doc.metadata)
else:
    print("No se encontraron documentos.")


675CHAPTER 118Pericardial Disease
 CHAPTER 118
■■ ECHOCARDIOGRAM
Thickened pericardium, normal ventricular contraction; abrupt halt in ven -
tricular filling after early diastole. Dilatation of IVC is typical. Dramatic effects 
of respiration: During inspiration the ventricular septum shifts to the left with 
prominent reduction of blood flow velocity across mitral valve; pattern reverses 
during expiration (opposite pattern across the tricuspid valve with respiration).
■■ CT OR MRI
More precise than echocardiogram for demonstrating thickened pericardium 
(present in >80% of pts with constrictive pericarditis).
■■ CARDIAC CATHETERIZATION
Equalization of diastolic pressures in all chambers; ventricular pressure tracings 
show “dip and plateau” appearance. Differentiate from restrictive cardiomyopa-
thy (Table 118-3).
TREATMENT
Constrictive Pericarditis
Surgical removal of the pericardium. Progressive improvement ensues over sev-
eral months.
APPROACH TO THE PATIENT
Asymptomatic Pericard

# **3. Document processing & spliting**
<hr style="height:5px;border-width:0;color:orange;background-color:orange">

Before injecting the documents to the RAG it is recomended to **normalize and clean** the text as the PDF reader might include some typos and errors that could distract the retrieval model.

Once normalized it'll be **chopped into smaler chunks** for the ingestion. This splitting can be done using different techniques such as**TokenSpliters**,**RecursiveTextSpliter**,or even a topic-related split (**BERTopic**) but once iterated over all of them the best results comes when using the **Semantic Chunker**, that will split the phrases according to an embedding meaningful sense. From there to generate a first split and from here another split into smaller chunks


## 3.2 Semantic Chunker: Embedding application as splitter
---

A simple and easy method is to split the document into chunks with the spliters available in Lanchain, but as it is very specific topic and my aim is to feed the RAG with a good context, I will use **SPubMedBert** which will split the chunks according to topic similarity.

From this initial split it'll be initialized the second split into smaller chunks with the spliters from Langchain(RecursiveCaracheters)

Now we will have smaller chunks more coherent in their content.

In [ ]:
# Specify the SentenceTransformer model name
SPubMedBert = "pritamdeka/S-PubMedBert-MS-MARCO"

# Initialize HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name=SPubMedBert)

# Configure the SemanticChunker with breakpoint threshold and chunk size constraints
text_splitter = SemanticChunker(
    embeddings=embeddings,
    buffer_size=1,
    breakpoint_threshold_type="percentile",  # Use percentile-based thresholds
    breakpoint_threshold_amount=73,         # Split points determined at 95th percentile similarity
    min_chunk_size=100,                     # Minimum chunk size in tokens
    sentence_split_regex=r"(?<=[.?!])\s+"   # Regex to split sentences
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/461k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# cleaning the text before the embeding generation.

def clean_text(text):
    # Remove excessive spaces and newlines
    text = re.sub(r'\s+', ' ', text)
    # Preserve alphanumeric characters, spaces, and symbols like + and - when associated with words/numbers
    text = re.sub(r'[^\w\s.,;:!?+-]', '', text)
    # Preserve patterns like chemical formulas (e.g., CO3-, K+), ranges (200-400), and units (+40mm)
    text = re.sub(r'(?<!\w)([-+])(?![\w])', '', text)  # Remove isolated + and -
    # Normalize case to lowercase
    text = text.lower()
    return text


In [15]:
# Store the chunked documents with metadata associated.
splits = []

for doc in docs:
    content = clean_text(doc.page_content)
    metadata = doc.metadata  # Preserve metadata

    if content:
        # Split the content into semantic chunks
        chunks = text_splitter.split_text(content)
        for chunk in chunks:
            # Create new Document objects for each chunk
            splits.append(Document(page_content=chunk, metadata=metadata))


In [16]:
# Output the number of chunks and an example chunk
print(f"Total number of chunks: {len(splits)}")
if splits:
    i= random.randint(50, 13000)
    example_chunk = splits[i]

    print(f"Example Chunk:\n{example_chunk.page_content}\nMetadata: {example_chunk.metadata}\n")
    print(f"Chunk number: {i}")

Total number of chunks: 15754
Example Chunk:
chronic exposure to ethanol is a leading cause of cirrhosis in humans, but the mechanism underlying the effect is the subject of considerable debate. malnutrition frequently accompanies alcoholism, and some investigators hypothesize that it is this factor, rather than the alcohol, that causes the cirrho  sis. evidence has been presented show  ing that rats that are maintained on an adequate diet can be exposed to ethanol without developing cirrhosis, but other studies have indicated that monkeys develop precirrhotic changes with expo  sure to ethanol even if no nutritional defi- ciencies develop.
Metadata: {'source': 'EN_principles-of-toxicology-third-edition.pdf', 'page': 254}

Chunk number: 8787


# **4. Embedding generation using a specialized Biomedical Domain Model**
<hr style="height:5px;border-width:0;color:orange;background-color:orange">

As the field of the documents are very specialized, it'll be needed a model focused and with expertise in a related field. Many clinical words and drug names will be better understood by a domain-specific model than a general-purpose model such as GPT 3.5-Turbo.

For this reason I iterated over some specific models such as BioBert, SciBERT and PubMedBert giving better results the last one.

**PubMedBert** is a model designed for **biomedical and clinical** text trained with data from PubMed abstracts and PMC text articles (~17Million documents)

PubMedBert is based in the BERT architecture having around **110M parameters**


This model will create the embedings needed to operate the retriever of the RAG. The retriever will look in a Vector-database (Chroma) and extract the most relevant chunks acording with our query


<p align="center">
    <img src="images/pubmedbert.png" alt="Descripción de la imagen" width="70%">
</p>



In [17]:

# name of SPubMedBert in Hugging face
SPubMedBert ="pritamdeka/S-PubMedBert-MS-MARCO"


# Initialize the sentence transformer and embeddings

embeddings = HuggingFaceEmbeddings(model_name=SPubMedBert)


# **5. RAG construction**
<hr style="height:5px;border-width:0;color:orange;background-color:orange">

RAG stands for Retrieval Augmentated Generation. In summary an embedding model is used to vectorize the information and later on search based on this Vector DataBase to provide to an LLM the context to answer from. This avoid the costly and heavy computational task of re-training a LLM into a certain domain.




<p align="center">
    <img src="images/rag_pipeline.png" alt="Descripción de la imagen" width="60%">
</p>



In [18]:

# Pass the embeddings to Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

print("Chroma vectorstore initialized with optimized PubMedBERT embeddings.")


Chroma vectorstore initialized with optimized PubMedBERT embeddings.


In [19]:
# Retrieve and generate using the relevant snippets from the books.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})


In [ ]:
# Let's check if the retriever is working correctly with the test text

retriever.invoke("Who won the US presidential elections in 2024?")


[Document(metadata={'source': 'Generated Text - Trump 2024'}, page_content='the 2024 elections saw the highest voter turnout in over 120 years, highlighting the deep political engagement of the american populace. '),
 Document(metadata={'source': 'Generated Text - Trump 2024'}, page_content=' in a surprising turn of events, donald j. trump won the 2024 us presidential elections. running as the republican candidate, trump secured 290 electoral votes, defeating the democratic candidate, kamala harris, who garnered 248 electoral votes.the victory was largely attributed to high voter turnout in swing states such as pennsylvania, michigan, and arizona. trumps campaign focused on economic recovery, with promises to lower inflation and bolster manufacturing jobs in the us.'),
 Document(metadata={'source': 'Generated Text - Trump 2024'}, page_content='key moments from the campaign included a record-breaking rally in texas and a strong debate performance where trump emphasized his america first

In [ ]:
# Visualization of an exaple of retrieved documents using a medical domain question

import textwrap

line_width = 80

# retrieved_docs = retriever.get_relevant_documents("What is the treatment for vaginal candidiasis?")
retrieved_docs = retriever.invoke("What is the treatment for vaginal candidiasis?")


for doc in retrieved_docs:
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Page: {doc.metadata.get('page', 'Unknown')}")
    print("Content:")
    print(textwrap.fill(doc.page_content, width=line_width))
    print("-" * line_width)


Source: EN_harrisons-manual-of-medicine.pdf
Page: 438
Content:
vaginalis is available. treatment vulvovaginal infections  vulvovaginal
candidiasis: miconazole a single 100-mg vaginal suppository, clotrimazole 100-mg
vaginal tablets daily for 37 days, or fluconazole 150 mg po once are all
effective. trichomoniasis: metronidazole 2 g po once or 500 mg po bid for 7 days
is effective.
--------------------------------------------------------------------------------
Source: EN_harrisons-manual-of-medicine.pdf
Page: 281
Content:
treatment candidiasis see also chap. 108. removal of predisposing factors;
topical nystatin or azoles; systemic therapy reserved for immunosuppressed pts,
unresponsive chronic or recurrent disease; vulvovaginal candidiasis may respond
to a single dose of flu- conazole, 150 mg. warts cutaneous neoplasms caused by
human papilloma viruses hpvs.
--------------------------------------------------------------------------------
Source: EN_harrisons-manual-of-medicine.pdf
Pag

# **6. Building the LLM for generate the answer**
<hr style="height:5px;border-width:0;color:orange;background-color:orange">

For the LLM model used for generation I choosed Mistral 7B as it is one of the most powerful LLM models to date and the resources compsumption and power seems ideal.

Another factor that help to decide is that is open source.

Mistral model will handle the context retrieved by the RAG and throught a Propmt will answer wisely.

In [22]:

# Define the model name
mistral = "mistralai/Mistral-7B-v0.1"

# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_compute_dtype="float16",  # Use fp16 for efficient computation
    bnb_4bit_use_double_quant=True,  # Use double quantization for memory efficiency
    bnb_4bit_quant_type="nf4"  # Use "nf4" (normal float 4) quantization
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(mistral)

# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    mistral,
    device_map="auto",  # Automatically map layers to CPU/GPU
    quantization_config=quantization_config  # Use the quantization configuration
)


# Create Hugging Face pipeline with adjusted parameters
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,  # Keep the inference in fp16 for efficiency
    max_new_tokens=512,        # Generate up to 512 tokens
    temperature=0.7,           # Control randomness
    top_p=0.9,                 # Nucleus sampling
    top_k=50,                  # Top-k sampling
    repetition_penalty=1.1     # Penalize repetitive sequences
)

# Wrap the pipeline in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


# **7. Testing**
<hr style="height:5px;border-width:0;color:orange;background-color:orange">

Showtime! Now let's check if the LLM (Mistral) is able to answer both previous asked questions correctly:

- What's the treatment for vaginal candidiasis?

- Who won US presidential elections in 2024?

The idea behind this is to check if the base-model is able to answer **medical questions precisely** giving a concise technical answer and in other way check if its able to answer correctly to an **event later than its trainning** (let's mention the training of Mistral 7B was made in Sept 2023)

In [23]:
llm_resp= llm.invoke('What is the treatment for vaginal candidiasis?')
print(llm_resp)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the treatment for vaginal candidiasis?

Vaginal candidiasis can be treated with antifungal medications. These include:

- Butoconazole (Gynazole)
- Clotrimazole (Canesten, Mycelex)
- Fluconazole (Diflucan)
- Miconazole (Monistat)
- Nystatin (Mycostatin)
- Terconazole (Terazol)

These medications are available in creams, ointments and suppositories that you insert into your vagina. They’re also available as a single oral dose of fluconazole.

How long does it take to treat vaginal candidiasis?

The length of time it takes to treat vaginal candidiasis depends on the type of medication used. Creams, ointments and suppositories usually need to be inserted once or twice daily for 3 to 7 days. A single dose of fluconazole may be taken by mouth.

What are the side effects of treating vaginal candidiasis?

Side effects from topical treatments for vaginal candidiasis are uncommon. Side effects from oral fluconazole include nausea, vomiting, headache, dizziness, diarrhea and abdominal pa

In [24]:
llm_resp_2= llm.invoke('Who won US presidential elections in 2024?')
print(llm_resp_2)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Who won US presidential elections in 2024?

The United States presidential election of 2024 will be the 59th quadrennial presidential election. It will be held on Tuesday, November 5, 2024. In this election, voters will select presidential electors who in turn will vote to elect a new president and vice president.

Who is running for president in 2024?

The 2024 Republican Party presidential primaries will be a series of electoral contests within the Republican Party to determine its nominee for President of the United States in the 2024 U.S. presidential election. The primaries are scheduled to begin in February 2024.

Who is running for president in 2024 UK?

The 2024 United Kingdom general election will be the next nationwide election of members of the House of Commons of the United Kingdom. It is expected to take place no later than 7 May 2024, four years after the previous election.



**TESTING**

--- 

We see the base-model is not able to give precise dosing information for example but in a general way is answering the question more or less precisely (it migth be hallucinating too).

Regarding the post-event question it correclty gives a non-binding answer proving that the answeer is out of his knowledge.

---

# **8. Building the RAG-Chain & Prompt Engineering**
<hr style="height:5px;border-width:0;color:orange;background-color:orange">

Once the LLM is set let's build the chain that will operate the whole pipeline. Langchain is well knwon for providing the perfect environement for LLM interactions. This chain will be integrated by:

- **Context:** provided by the RAG retriever according to a certain query.

- **Propmt**: Prompt injected as input for the LLM model giving details about its main task, how to behave, inputs and any rules we want to it to follow. This prompt will be key to addapt the LLM output to our requierements. 

- **LLM:** Mistral 7B, in charge of providing the generative content.



In [ ]:

# Prompt
template = """
You are a medical assistant with expertise in interpreting technical medical documents and answering complex questions. Based on a Retrieval-Augmented Generation(RAG) a particular context will be given for answering the question.

Your task is to provide concise, accurate, and well-supported answers to questions using the retrieved context by the RAG to enhance your responses.
If the retrieved context is insufficient, provide your best answer based on your medical expertise and clarify any uncertainties.

### Instructions:
1. Analyze the question carefully.
2. Use the provided context to support your answer.
3. If additional relevant information is required but missing, mention this explicitly and provide an informed response based on your expertise.
4. Always include references to the provided context (e.g., document source and page) to substantiate your answer.

### Formatting:
- Begin with a direct answer to the question.
- Follow up with additional details, citing the relevant retrieved context.
- If applicable, clarify the reliability or gaps in the provided context.

### Input:
Question to be answered: {question}
Retrieved Context:
{context}

"""
prompt= ChatPromptTemplate.from_template(template)

# LLM
# already defined

# Chain
chain= (
    {"context":retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    |StrOutputParser()
)


In [ ]:
# let's check the output of the chain with the question "What is the treatment for vaginal candidiasis?"
chain_out= chain.invoke("What is the treatment for vaginal candidiasis?")
print(chain_out)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Human: 
You are a medical assistant with expertise in interpreting technical medical documents and answering complex questions. Based on a Retrieval-Augmented Generation(RAG) a particular context will be given for answering the question. 

Your task is to provide concise, accurate, and well-supported answers to questions using the retrieved context by the RAG to enhance your responses. 
If the retrieved context is insufficient, provide your best answer based on your medical expertise and clarify any uncertainties.

### Instructions:
1. Analyze the question carefully.
2. Use the provided context to support your answer.
3. If additional relevant information is required but missing, mention this explicitly and provide an informed response based on your expertise.
4. Always include references to the provided context (e.g., document source and page) to substantiate your answer.

### Formatting:
- Begin with a direct answer to the question.
- Follow up with additional details, citing the rel

In [ ]:
# and the testing of the chain with the question "Who won the 2024 US presidential elections?"
chain_out_2= chain.invoke("Who won the 2024 US presidential elections?")
print(chain_out_2)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Human: 
You are a medical assistant with expertise in interpreting technical medical documents and answering complex questions. Based on a Retrieval-Augmented Generation(RAG) a particular context will be given for answering the question. 

Your task is to provide concise, accurate, and well-supported answers to questions using the retrieved context by the RAG to enhance your responses. 
If the retrieved context is insufficient, provide your best answer based on your medical expertise and clarify any uncertainties.

### Instructions:
1. Analyze the question carefully.
2. Use the provided context to support your answer.
3. If additional relevant information is required but missing, mention this explicitly and provide an informed response based on your expertise.
4. Always include references to the provided context (e.g., document source and page) to substantiate your answer.

### Formatting:
- Begin with a direct answer to the question.
- Follow up with additional details, citing the rel

# **9. Conclusion**
<hr style="height:5px;border-width:0;color:orange;background-color:red">

Based on the executed lines the **LLM(Mistral 7B)** without the implementation of the RAG technique is not able to answer correclty the question about US Presidential elections as we expected but when associated with the RAG, it provide the precise and accurate answer desired proving thad the RAG is working well.

Regarding the medical output is yet to be determined if this is providing the correct and precise healthcare answer always, but it can search information precisely and support its response with that ''evidence'' which I believe is a much more robust answer than solely the LLM output (very imprecise and general.).

The **RAG-chain** created is able to provide **dosing and forms of application** as well as the **retrieved context** to check manually if needed giving a much more robust and precise answer.

And as in the fairy tales, this story ends. **And they lived happily ever after.**

Thanks for watching!


<p align="center">
    <img src="images/wedding.jpg" alt="Descripción de la imagen" width="60%">
</p>